In [185]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import matplotlib.pyplot as plt

In [186]:
ws_buffer = 1025
RWS_buffer = 1000

In [187]:
#load data
wsgrenzen = r"../../../../Data_overig/Waterschapsgrenzen/waterschap.gpkg"
RWSgrenzen = r"../../../../Data_overig/Waterschapsgrenzen/Rijkswaterstaat.gpkg"

wsgrenzen = gpd.read_file(wsgrenzen)
RWSgrenzen = gpd.read_file(RWSgrenzen)


In [188]:
#clean df and names
wsgrenzen = wsgrenzen[['naam', 'geometry']]

# Removing "\n", "waterschap", "Hoogheemraadschap", "van" and spaces and commas
wsgrenzen['naam'] = wsgrenzen['naam'].str.replace(r'\n', '', regex=True)
wsgrenzen['naam'] = wsgrenzen['naam'].str.replace('Waterschap', '', regex=False)
wsgrenzen['naam'] = wsgrenzen['naam'].str.replace('Hoogheemraadschap', '', regex=False)
wsgrenzen['naam'] = wsgrenzen['naam'].str.replace('De S', 'S', regex=False) #HDSR
wsgrenzen['naam'] = wsgrenzen['naam'].str.replace('â', 'a', regex=False) #WF
wsgrenzen['naam'] = wsgrenzen['naam'].str.replace('van', '', regex=False)
wsgrenzen['naam'] = wsgrenzen['naam'].str.replace(',', '', regex=False)
wsgrenzen['naam'] = wsgrenzen['naam'].str.replace("'", '', regex=False)
wsgrenzen['naam'] = wsgrenzen['naam'].str.replace(' ', '', regex=False)

wsgrenzen = wsgrenzen.sort_values(by = 'naam').reset_index(drop=True)
wsgrenzen_OG = wsgrenzen.copy()


In [189]:
#get rid of irrelvant polygons
wsgrenzen = wsgrenzen.explode()
wsgrenzen['area'] = wsgrenzen.area
wsgrenzen = wsgrenzen.loc[wsgrenzen.area > 10000000] #remove some small polygons
wsgrenzen.reset_index(drop=True, inplace=True)

/tmp/ipykernel_2272423/193568770.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  wsgrenzen = wsgrenzen.explode()


In [190]:
#add RWSgrenzen. Buffer and dissolve it
RWSgrenzen['geometry'] = RWSgrenzen.buffer(RWS_buffer)
RWSgrenzen = RWSgrenzen.dissolve()[['geometry']]
# RWSgrenzen['naam'] = 'Rijkswaterstaat'

In [191]:
# Remove the RWS area in each WS
wsgrenzen_cut_out = gpd.overlay(wsgrenzen, RWSgrenzen, how='symmetric_difference')
wsgrenzen_cut_out.dropna(inplace=True)

/tmp/ipykernel_2272423/1225746930.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 996 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  wsgrenzen_cut_out = gpd.overlay(wsgrenzen, RWSgrenzen, how='symmetric_difference')


In [192]:
#add a name to the RWS area
RWSgrenzen['naam'] = 'Rijkswaterstaat'

In [193]:
#add a buffer to each waterschap. Within this strip an authority will be found.
wsgrenzen_cut_out['geometry'] = wsgrenzen_cut_out.buffer(ws_buffer)

In [194]:
#add the two layers together
authority_borders = pd.concat([wsgrenzen_cut_out, RWSgrenzen])
authority_borders = authority_borders.reset_index(drop=True)
authority_borders = gpd.GeoDataFrame(authority_borders, geometry = 'geometry').set_crs(crs='EPSG:28992')

In [195]:
# Add a bit more area by dissolving it with the original gdf
authority_borders = pd.concat([authority_borders, wsgrenzen_OG])
authority_borders = gpd.GeoDataFrame(authority_borders, geometry = 'geometry').set_crs(crs='EPSG:28992')
authority_borders = authority_borders.dissolve(by='naam', as_index=False)

In [197]:
#inspect results
# authority_borders[['naam', 'geometry']].to_file('authority_borders.gpkg', driver='GPKG')

In [196]:
authority_borders

,naam,geometry,area
0,AaenMaas,"POLYGON Z ((200743.120 396718.157 0.000, 20077...",1.609866e+09
1,AmstelGooienVecht,"POLYGON Z ((149117.970 476526.468 0.000, 14908...",7.742425e+08
2,BrabantseDelta,"POLYGON Z ((133132.687 414182.598 0.000, 13313...",1.707697e+09
3,DeDommel,"POLYGON Z ((130995.686 399274.178 0.000, 13100...",1.510061e+09
4,Delfland,"POLYGON Z ((91404.790 436037.970 0.000, 91405....",4.475579e+08
5,DrentsOverijsselseDelta,"POLYGON Z ((188233.552 502597.219 0.000, 18817...",2.548828e+09
6,HollandsNoorderkwartier,"POLYGON Z ((122684.109 488664.375 0.000, 12267...",3.133110e+09
7,HollandseDelta,"POLYGON Z ((92759.975 435579.270 0.000, 93348....",1.436512e+09
8,HunzeenAas,"POLYGON Z ((227631.170 549040.190 0.000, 22763...",2.073997e+09
9,Limburg,"POLYGON Z ((172715.341 318817.654 0.000, 17272...",2.209850e+09
